### PLAN
1. Load up to Jun 2022 into BigQuery
2. Create github action to run BQ insert script nightly
3. Set grouped database table (history_df) in BigQuery to refresh nightly after insert
4. Make script to pull data history_df from BigQuery data

8. For streamlit app append [history_df, current_df, future_df] to make the combo_df
    - put history_df into cache using st.experimental_memo so its only called once each time app is opened

In [4]:
# Initialize python libs & SQL creads
from datetime import datetime, timedelta, date
import pandas as pd
from pandasql import sqldf
import streamlit as st
import pull_nrg_data
import json
import http.client
import certifi
import ssl
import os
import timeit
import pickle

In [ ]:
pull_nrg_data.release_token(accessToken)

In [150]:
# Release token
import certifi
import ssl
import http.client
server = 'api.nrgstream.com'

def release_token(accessToken):
    path = '/api/ReleaseToken'
    headers = {'Authorization': f'Bearer {accessToken}'}
    context = ssl.create_default_context(cafile=certifi.where())
    conn = http.client.HTTPSConnection(server,context=context)
    conn.request('DELETE', path, None, headers)
    res = conn.getresponse()
    print('token released')

accessToken = 'Bvmk95JWlxZKiXSFb41jCiKlG_96XbHxb_ZSSeFECiWgFk_Y2kBMS0-5gitwoith_1Uvcy_MpZ7yHcpivRX69UJeq_jjeoYYvfqhiMNEAZCAzy_JWjBMuVmrxZFl_P1WfbBSJXFNDrHK2WPIfwCfCQ4FF4YjOx7CI2OrQgN3VvfwnyPXf1Yuujqhfj0oV91YYPixIcI-YdTHOjIpRX34vutUR9pcDd4WX2hsid9_yGC_IGPfXuy_M7PKq7KGvXwCrj-Twdz8pkNIfXkdcAl1mRlt5Lgd4qQdFYBtknqW3jqm12joed1gBECGtuC6G2sQN5nOMA3eyrXeo5oNVfGEngrGsvs'
release_token(accessToken)

token released


In [ ]:
# Joining daily to hourly data
import pandas as pd

intraday = pd.DataFrame(index=pd.date_range('2016-01-01', '2016-01-07', freq='H'),data=[i for i in range(145)], columns=['hourly'])
daily = pd.DataFrame(index=pd.date_range('2016-01-01', '2016-01-07', freq='D'), data=[i for i in range(7)], columns=['daily'])
df = intraday.join(daily).fillna(method='ffill')
df

In [147]:
with open('./alerts.pickle', 'rb') as handle:
        alerts = pickle.load(handle)
alerts

{'dates': {'Biomass & Other': datetime.datetime(2022, 1, 1, 0, 0),
  'Coal': datetime.datetime(2022, 1, 1, 0, 0),
  'Dual Fuel': datetime.datetime(2022, 1, 1, 0, 0),
  'Energy Storage': datetime.datetime(2022, 1, 1, 0, 0),
  'Hydro': datetime.datetime(2022, 1, 1, 0, 0),
  'Natural Gas': datetime.datetime(2022, 8, 30, 10, 23, 10, 345549),
  'Solar': datetime.datetime(2022, 1, 1, 0, 0),
  'Wind': datetime.datetime(2022, 1, 1, 0, 0)},
 'dataframes': [(datetime.date(2022, 8, 30),
       timeStamp  value         fuelType
   0  2022-01-01    700             Coal
   1  2022-02-01      0             Coal
   2  2022-03-01     80             Coal
   3  2022-04-01    310             Coal
   4  2022-05-01      0             Coal
   ..        ...    ...              ...
   3  2024-04-01     30  Biomass & Other
   4  2024-05-01     30  Biomass & Other
   5  2024-06-01     30  Biomass & Other
   6  2024-07-01     30  Biomass & Other
   7  2024-08-01     30  Biomass & Other
   
   [256 rows x 3 column

In [144]:
with open('./alerts.pickle', 'wb') as handle:
    pickle.dump(alerts, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [148]:
alerts['dataframes'][0][1]

,timeStamp,value,fuelType
0,2022-01-01,700,Coal
1,2022-02-01,0,Coal
2,2022-03-01,80,Coal
3,2022-04-01,310,Coal
4,2022-05-01,0,Coal
...,...,...,...
3,2024-04-01,30,Biomass & Other
4,2024-05-01,30,Biomass & Other
5,2024-06-01,30,Biomass & Other
6,2024-07-01,30,Biomass & Other


In [139]:
for i in range(7):
    alerts['dataframes'][i] = (datetime.now().date(), df)
    

[0, 0, 0, 0, 0, 0, 0]